In [47]:
# environment set up
import keras
from keras.models import Sequential
from keras.layers import Reshape, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import to_categorical
import tensorflow as tf # only use tensorflow keras
import numpy as np
import matplotlib.pyplot as plt
import os
import random

# working folder
directory = "/Users/Srikar/Desktop/Velexi/spectra-ml/data"
os.chdir(directory)

# print(os.getcwd())

In [2]:
dataset = str(directory + "/dataset3")
num_samples = len([name for name in os.listdir(dataset) if \
                   os.path.isfile(os.path.join(dataset,name))])

print(num_samples)

33


In [3]:
spectrum_len = 480
spectra = np.zeros((num_samples,spectrum_len))

spectrum_categories = np.zeros(num_samples)
first_record_of_mixtures_chapter = 11602
is_a_mineral = 1                                   # these numbers match the chapter numbers given by usgs
is_a_mixture = 2
spectrum_names = ["" for x in range(num_samples)]

y = np.zeros((num_samples, 1))

os.chdir(dataset)

i = 0

for filename in os.listdir(dataset):
    file_object  = open(filename, 'r').readlines()
#     strip off header, add to matrix 'spectra'
    spectra[i,:] = file_object[1:]

#     label spectrum class, based on header
#     actinolite: 0, alunite: 1, chlorite: 2
    material_name = file_object[0]
    
    spectrum_names[i] = material_name
    
    start = 'Record='
    end = ':'
    record_number = int((material_name.split(start))[1].split(end)[0])
    # print(record_number)
    if record_number < first_record_of_mixtures_chapter:
        spectrum_categories[i] = is_a_mineral
    else:
        spectrum_categories[i] = is_a_mixture

#     print(material_name)

    if material_name.find('Actinolite',) != -1: # if material name contains actinolite
        y[i,0] = 0
    elif material_name.find('Alun',)!= -1:
        y[i,0] = 1
    else: # chlorite
        y[i,0] = 2

#     turn missing points into 0
    for j in range(spectrum_len):
        if spectra[i,j] < 0:
            spectra[i,j] = 0
    i+=1

In [68]:
sample_indices = list(range(0, num_samples))
print(num_samples)
random.shuffle(sample_indices)
train_set_size = 3*(num_samples//5)
dev_set_size = (num_samples//5)
test_set_size= num_samples-dev_set_size - train_set_size
print(train_set_size)
print(test_set_size)
print(dev_set_size)
train_set_indices = sample_indices[:train_set_size]
dev_set_indices = sample_indices[train_set_size: train_set_size+dev_set_size]
test_set_indices= sample_indices[train_set_size+dev_set_size: num_samples]
print(train_set_indices)
print(test_set_indices)
print(dev_set_indices)

train_set = spectra[train_set_indices, :]
train_labels = y[train_set_indices, :]
dev_set = spectra[dev_set_indices, :]
dev_labels = y[dev_set_indices, :]
test_set = spectra[test_set_indices, :]
test_labels = y[test_set_indices, :]

33
18
9
6
[9, 15, 1, 17, 10, 30, 20, 12, 28, 21, 22, 16, 26, 0, 29, 23, 14, 8]
[6, 32, 24, 3, 27, 7, 25, 18, 13]
[4, 5, 31, 2, 11, 19]


In [69]:
train_labels = train_labels.flatten()
dev_labels = dev_labels.flatten()
test_labels = test_labels.flatten()
# type(train_labels)

In [70]:
# len(train_set)
# len(train_set[17])
print(test_set.shape)

(9, 480)


In [71]:
# --- plot the classes

# plot each class in a separate plot
# plot spectra names in legend
# plot minerals and mixtures w diff line widths

mineral_names = ["Actinolite", "Alunite", "Chlorite"]

# variables
num0 = 0 #number of samples of class 0
num1 = 0
num2 = 0

mineral_linewidth = 1         # linewidth = 1 is default
mixture_linewidth = 3         

# count the number of each class to make spectra0, spectra1, spectra2 databases
for i in range(num_samples):
    if y[i,0]== 0:
        num0 += 1
    elif y[i,0]== 1:
        num1 += 1
    elif y[i,0]== 2:
        num2 += 1

# make class-specific databases spectra0, ...1, ...2
spectra0 = np.zeros((num0,spectrum_len)) 
spectra1 = np.zeros((num1,spectrum_len)) 
spectra2 = np.zeros((num2,spectrum_len)) 

labels0 = ["" for x in range(num0)]
labels1 = ["" for x in range(num1)]
labels2 = ["" for x in range(num2)]

linewidth0 = np.zeros(num0)
linewidth1 = np.zeros(num1)
linewidth2 = np.zeros(num2)


# make counters for each database to place spectra
i0 = 0
i1 = 0
i2 = 0

# set linewidth for the spectrum 
# populate class-specific databases spectra0, ...1, ...2
for i in range(num_samples):
    
    # set linewidth
    #testcode
    #print(spectrum_categories)
    #print(spectrum_categories[i])
    
    if spectrum_categories[i] == is_a_mineral:
        linewidth = mineral_linewidth
        
        #testcode
        #print('min')
    else: 
        linewidth = mixture_linewidth
        
        #testcode
        #print('mix')
    
    # populate matrices for making each class plot
    if y[i,0]== 0:
        spectra0[i0,:] = spectra[i,:]
        labels0[i0] = spectrum_names[i]
        linewidth0[i0] = linewidth
        i0 +=1
    elif y[i,0]== 1:
        spectra1[i1,:] = spectra[i,:]
        labels1[i1] = spectrum_names[i]
        linewidth1[i1] = linewidth
        i1 +=1
    else:
        spectra2[i2,:] = spectra[i,:]
        labels2[i2] = spectrum_names[i]
        linewidth2[i2] = linewidth
        i2 +=1

# plot each class-specific database separately
# for i in range(i0):
#     fig = plt.figure()
#     plt.plot(range(1, spectrum_len+1), spectra0[i,:], label = labels0[i], linewidth = linewidth0[i]) # remove linewidth for all mixtures/minerals to be standard
#     plt.xticks([])
#     plt.yticks([])
# #     plt.show()
#     path = "/Users/Srikar/Desktop/Velexi/spectra-ml/lab-notebook/smunukutla/plots/" + mineral_names[0] + str(i) + ".png"
#     fig.savefig(path, format = "PNG")
# # plt.legend(bbox_to_anchor=(1.1, 1.05))

# for i in range(i1):
#     plt.plot(range(1, spectrum_len+1), spectra1[i,:], label = labels1[i], linewidth = linewidth1[i])
# plt.legend(bbox_to_anchor=(1.1, 1.05))
# plt.show()

# for i in range(i2):
#     plt.plot(range(1, spectrum_len+1), spectra2[i,:], label = labels2[i], linewidth = linewidth2[i])
# plt.legend(bbox_to_anchor=(1.1, 1.05))
# plt.show()

In [72]:
train_set = np.reshape(train_set, (train_set.shape[0], 480, 1))
dev_set = np.reshape(dev_set, (dev_set.shape[0], 480, 1))
test_set = np.reshape(test_set, (test_set.shape[0], 480, 1))

train_labels = np.reshape(train_labels, (train_labels.shape[0], 1))
dev_labels = np.reshape(dev_labels, (dev_labels.shape[0], 1))
test_labels = np.reshape(test_labels, (test_labels.shape[0], 1))

train_labels = to_categorical(train_labels)
dev_labels = to_categorical(dev_labels)
test_labels = to_categorical(test_labels)

In [73]:
print(train_labels)

[[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [74]:
model = Sequential()
# model.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
model.add(Conv1D(64, 25, activation='relu', input_shape=(train_set.shape[1], 1)))
model.add(Conv1D(64, 25, activation='relu'))
model.add(MaxPooling1D(4)) # 108 by 64 so far
model.add(Conv1D(100, 25, activation='relu'))
model.add(Conv1D(100, 25, activation='relu'))
model.add(MaxPooling1D(4))
# model.add(Dropout(0.5))
# model.add(GlobalAveragePooling1D())
model.add(Flatten())
model.add(Dense(3, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_61 (Conv1D)           (None, 456, 64)           1664      
_________________________________________________________________
conv1d_62 (Conv1D)           (None, 432, 64)           102464    
_________________________________________________________________
max_pooling1d_27 (MaxPooling (None, 108, 64)           0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 84, 100)           160100    
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 60, 100)           250100    
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 15, 100)           0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 1500)              0         
__________

In [75]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 12
EPOCHS = 50

print(train_labels.shape)
model.fit(train_set, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(test_set, test_labels)) 

(18, 3)
Train on 18 samples, validate on 9 samples
Epoch 1/50
18/18 [==============================] - 1s 62ms/step - loss: 1.0620 - acc: 0.4444 - val_loss: 0.9794 - val_acc: 0.4444
Epoch 2/50
18/18 [==============================] - 0s 12ms/step - loss: 1.0196 - acc: 0.3889 - val_loss: 0.9352 - val_acc: 0.4444
Epoch 3/50
18/18 [==============================] - 0s 14ms/step - loss: 0.9504 - acc: 0.3889 - val_loss: 0.8943 - val_acc: 0.4444
Epoch 4/50
18/18 [==============================] - 0s 13ms/step - loss: 0.9356 - acc: 0.3889 - val_loss: 0.8768 - val_acc: 0.4444
Epoch 5/50
18/18 [==============================] - 0s 21ms/step - loss: 0.8831 - acc: 0.3889 - val_loss: 0.8884 - val_acc: 0.4444
Epoch 6/50
18/18 [==============================] - 0s 21ms/step - loss: 0.8833 - acc: 0.3889 - val_loss: 0.8436 - val_acc: 0.4444
Epoch 7/50
18/18 [==============================] - 0s 24ms/step - loss: 0.8257 - acc: 0.6667 - val_loss: 0.8169 - val_acc: 0.5556
Epoch 8/50
18/18 [==============